# Calculate Harmonic Parameters

Besides using static harmonic parameters to derive the non-flood likelihood $P(\sigma^0|nonflood)$, we can also dynamically calculate the parameters form past backscattering information. In this notebook we show how we can extract these coefficients that describe seasonal patterns in Sentinel 1 radar backscatter variability.

In [ ]:
import hvplot.xarray  # noqa
import numpy as np
import xarray as xr
from dask.distributed import Client
from dask_flood_mapper import flood
from dask_flood_mapper.catalog import (format_datetime_for_xarray_selection,
                                       initialize_catalog, initialize_search)
from dask_flood_mapper.harmonic_params import (
    create_harmonic_parameters, process_harmonic_parameters_datacube)
from dask_flood_mapper.processing import prepare_dc, process_sig0_dc

In [ ]:
client = Client(processes=False, threads_per_worker=2, n_workers=1, memory_limit="12GB")
client

As an example we will select only a small region of interest contained in the Zingst case study. 

In [ ]:
time_range = "2022-10-11T05:25:26"
minlon, maxlon = 12.999, 13
minlat, maxlat = 53.999, 54
bounding_box = [minlon, minlat, maxlon, maxlat]

We will now load the sigma nought datacube from EODC. In order to fit the harmonic functions to the timeseries, the temporal range is extended to include the three years before the provided time range. This is done by using the `dynamic` parameter of `initialize_search`.

In [ ]:
eodc_catalog = initialize_catalog()
search = initialize_search(eodc_catalog, bounding_box, time_range, dynamic=True)
items_sig0 = search.item_collection()
sig0_dc = prepare_dc(items_sig0, bounding_box, bands="VV")
sig0_dc, orbit_sig0 = process_sig0_dc(sig0_dc, items_sig0, bands="VV")
sig0_dc

## Calculate Harmonic Parameters

This function fits sine and cosine functions known as harmonic oscillators to each pixel of the Sentinel 1 $\sigma^0$ datacube. These seasonally varying curves can then be extracted from time series. What is left is the noise or transient events, for example flood events, superimposed on the seasonal trend.

In [ ]:
# Function to reformat time range for selecting from the Xarray
datetime_xr = format_datetime_for_xarray_selection(search, time_range)
hpar_list = create_harmonic_parameters(sig0_dc)
__, hpar_dc, __ = process_harmonic_parameters_datacube(sig0_dc, datetime_xr, hpar_list)
hpar_dc

In [ ]:
t = sig0_dc.time.dt.dayofyear
n = 365
y = (
    hpar_dc.M0
    + hpar_dc.C1 * np.cos(2 * np.pi * t / n)
    + hpar_dc.S1 * np.sin(2 * np.pi * t / n)
    + hpar_dc.C2 * np.cos(2 * np.pi * t / n)
    + hpar_dc.S2 * np.sin(2 * np.pi * t / n)
    + hpar_dc.C3 * np.cos(2 * np.pi * t / n)
    + hpar_dc.S3 * np.sin(2 * np.pi * t / n)
)

## Fit Harmonic Function to Original Data

Finally, we merge the two datasets and superimpose the fitted harmonic function on the raw sigma nought data. 

In [ ]:
xr.merge([y.rename("pred"), sig0_dc]).squeeze().assign_coords(
    x=range(len(sig0_dc.x)), y=range(len(sig0_dc.y))
).hvplot(x="time")

Calculating of harmonic parameters can be selected during flood mapping by setting the `parameter` to `True`. 

In [ ]:
flood.decision(bbox=bounding_box, datetime=time_range, dynamic=True)

This setting theoretically frees the user from the need for a precomputed harmonic parameters dataset, thereby only requiring sentinel-1 sigma nought and PLIA datasets as static inputs. Do mind, however, that calculating harmonic parameters can be a computationally expensive operation.